# 銀イルカの3人のセリフをスクレイピングして，同じところと違うところを

とりま wikiとかから智絵里のセリフをスクレイピングする． 

テキスト解析 -> 出現頻度，


セリフをword2vecで学習して，分散表現を計算できるようにする

分散表現を次元削減して2次元座標上にプロットすることで，単語ごとの類似度を可視化する

銀イルカ3人のセリフをスクレイピング  

doc2vecで学習して，

In [1]:
import re
import time
import random
import requests
from bs4 import BeautifulSoup
import pandas as pd

/home/wani/dere/idol_info/.venv/lib/python3.7/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
/home/wani/dere/idol_info/.venv/lib/python3.7/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
pd.options.display.max_rows = 500
pd.options.display.max_columns = 20 

## デレステセリフのスクレイピング

In [3]:
def get_quotes_sute(q):
    """
    デレステで公開されている各アイドルのセリフデータを取得する
    [arg]
    q: int
    [ret]
    df: DataFrame
    """
    html = f"https://starlight.kirara.ca/char/{q}"
    r = requests.get(html)
    r.encoding = r.apparent_encoding
    soup = BeautifulSoup(r.text, "lxml")
    texts = soup.find_all("span")
    corpus = []
    for text in texts:
        corpus.append(text.text)
    df = pd.DataFrame(corpus)
    #df[0].value_counts().head(10) # 値の出現頻度をカウント
    df = df[~df.duplicated()] # 重複行をdataframeから除外
    df = df[~df[0].str.match("^[\s|a-zA-Z|<|\+|\d$]")] # 正規表現にマッチするものを除外
    df = df[df[0] != ""]
    return df.reset_index(drop=True)

## モバマスセリフのスクレイピング

In [4]:
def get_quotes_moba(query="緒方智絵里", debug=False):
    """
    モバマスで公開されているセリフ（含 アイドル名、カード名 など）
    をスクレイピングして pandas dataframe にする
    
    [args]
    query: str 省略すると 緒方智絵里 のセリフをスクレイピングする
    debug: Bool Trueにするとスクレイピング数を上限2にする。それだけ もっと良い使い方あるはず...
    [ret]
    df: DataFrame
    """
    
    html = f"https://icws.indigo-bell.com/search?q={query}&st=n"
    r = requests.get(html)
    r.encoding = r.apparent_encoding
    soup = BeautifulSoup(r.text, "lxml")
    a = soup.find("a", string="Last　»").get("href")
    if debug == True:
        n = 2
    else:
        n = int(re.search(r"page=(\d+)&", a).group(1)) # クロールするページ数
    
    df = pd.DataFrame()
    for i in range(n): # セリフが登録されてるページ分ループする
        print(f"crawling page: {i + 1}/{n}")
        html = f"https://icws.indigo-bell.com/search?page={i + 1}&q={query}&st=n"
        r = requests.get(html)
        r.encoding = r.apparent_encoding
        soup = BeautifulSoup(r.text, "lxml")
        table = soup.find_all("tbody",class_="result")[0]
        rows = table.find_all("tr")
        tmp = []
        tm = 5 + random.randint(0, 5)

        for row in rows:
            cell_txt = []
            for cell in row.find_all(["td", "th"]):
                cell_txt.append(cell.text)
            tmp.append(cell_txt)
        df = df.append(tmp)
        #print(f"sleep: {tm}")
        if i != (n - 1):
            time.sleep(tm)
        else:
            continue
    return df.reset_index(drop=True) # indexを振り直してもとのindexも削除

In [5]:
# deresute id
# chieri : 112
# minami : 185
# yui    : 246
# sae    : 132
# hotaru : 161
# yoshino: 262

In [6]:
aa = get_quotes_sute(112)

In [7]:
bb = get_quotes_moba(query="緒方智絵里", debug=True)

crawling page: 1/2
crawling page: 2/2


In [8]:
aa.head(5)

,0
0,アイドルマスター シンデレラガールズ スターライトステージ!
1,お、おはようございます。アーリーモーニングティー、いかがですか…？
2,お昼だけど…もうちょっとだけ…午後のレッスンの予習をしなくっちゃ…
3,ふぁぁ…こんな時間までお疲れさまです。ホットミルク、飲みますか…？
4,プロデューサーさん…もしお暇なら、私…ステージに、立ちたい…です


In [56]:
bb.head(10)

,0,...
0,緒方智絵里,...
1,緒方智絵里,...
2,緒方智絵里,...
3,緒方智絵里,...
4,緒方智絵里,...
5,緒方智絵里,...
6,緒方智絵里,...
7,緒方智絵里,...
8,緒方智絵里,...
9,緒方智絵里,...


## モバ，ステの結合と正規化

In [20]:
bb[3] = bb[3].str.replace("○○", "PRODUCER") # OO (プロデューサー名)部分を 置換して統一
# aa + bb のコーパスを作る
docs = pd.Series()
docs = pd.concat([aa[0], bb[3]])
docs = docs.str.replace("プロデューサー", "PRODUCER")
docs = docs.str.replace("[Producer]", "PRODUCER", regex=False)
docs = docs.reset_index(drop=True)

## 形態素解析

mecabを使う。インストールしていなければめかぶをインストール

neeologdまでインストールする手順はの参考は
- [ubuntu 18.10 に mecab をインストール](https://qiita.com/ekzemplaro/items/c98c7f6698f130b55d53)
- [WSLでMeCabとJupyter Notebookを動かす](https://qiita.com/hironobukawaguchi3/items/61a8664dd26c1da66880)
- [MecabをPythonで使うまで](https://qiita.com/Sak1361/items/47e9ec464ccc770cd65c)

---

$ mecab -d /usr/lib/mecab/dic/mecab-ipadic-neologd

In [50]:
import MeCab
text = "テキストを形態素解析して分かち書きする"
print(extractNoun(text))

sur 
met ['BOS/EOS', '*', '*', '*', '*', '*', '*', '*', '*']
sur テキスト
met ['名詞', '一般', '*', '*', '*', '*', 'テキスト', 'テキスト', 'テキスト']
sur を
met ['助詞', '格助詞', '一般', '*', '*', '*', 'を', 'ヲ', 'ヲ']
sur 形態素解析
met ['名詞', '固有名詞', '一般', '*', '*', '*', '形態素解析', 'ケイタイソカイセキ', 'ケイタイソカイセキ']
sur し
met ['動詞', '自立', '*', '*', 'サ変・スル', '連用形', 'する', 'シ', 'シ']
sur て
met ['助詞', '接続助詞', '*', '*', '*', '*', 'て', 'テ', 'テ']
sur 分かち書き
met ['名詞', '一般', '*', '*', '*', '*', '分かち書き', 'ワカチガキ', 'ワカチガキ']
sur する
met ['動詞', '自立', '*', '*', 'サ変・スル', '基本形', 'する', 'スル', 'スル']
sur 
met ['BOS/EOS', '*', '*', '*', '*', '*', '*', '*', '*']
['テキスト', '形態素解析', '分かち書き']


In [84]:
def extractNoun(text, mecab_mode, noun="名詞"):
    """
    [arg]: sentence
    [ret]: a list of noun words
    """
    tagger = MeCab.Tagger(mecab_mode)
    node = tagger.parseToNode(text)
    keywords = []
    while node:
        surface = node.surface # 表層形を表示(str)
        meta = node.feature.split(",") # ['記号', '読点', '*', '*', '*', '*', '、', '、', '、']
        if meta[0] == noun:
            keywords.append(node.surface)
        node = node.next # 次のノードのアドレスを指定
    return keywords

def tokenize(text, wakati=True, dic="neologd"):
    """
    テキストを形態素解析して分かち書きする
    [arg]
    text: str
    [ret]
    term: str
    """
    import MeCab
    if dic == "neologd":
        mecab_md = MeCab.Tagger(r'-Owakati -d /usr/lib/mecab/dic/mecab-ipadic-neologd') # debugging
    elif dic == "ipa":
        mecab_mode = MeCab.Tagger(r'-Owakati')
    if wakati == True: # 分かち書きしたものをそのまま取り出す
        term = mecab_mode.parse(text).strip()
    else: # 品詞で
        term = extractNoun(text, mecab_mode)
    return term

In [22]:
print(f'ipa-dic    ： {tokenize(docs[99], dic = "ipa")}')
print(f'neologd-dic： {tokenize(docs[99])}')

ipa-dic    ： あっ 、 PRODUCER さん 、 今日 の わたし の お 仕事 、 どう でし た か ？
neologd-dic： あっ 、 PRODUCER さん 、 今日 の わたし の お仕事 、 どう でし た か ？


In [24]:
from gensim import corpora, models

In [27]:
docs_list = [tokenize(row).split() for row in docs]
dic = corpora.Dictionary(docs_list)
dic.save_as_text('./data/text.dict')  # 保存
#gensim.corpora.Dictionary.load_from_text('./data/text.dict')  # 次回からファイルロード可

In [28]:
print(len(dic)) # dictionaryに登録された単語(形態素)数

1805


In [32]:
# 単語出現頻度を数える
# 余談: bashだと $ sort -t$'\t' -k3 -nr text.dict で単語とその出現頻度回数カウントできる
from collections import defaultdict
freq = defaultdict(int)

for text in docs_list:
    for token in text:
        freq[token] += 1
print(sorted(freq.items(), key=lambda x:x[1], reverse=True))

In [39]:
corpus = [dic.doc2bow(doc) for doc in docs_list] # コーパスの作成
corpora.MmCorpus.serialize('./data/text.mm', corpus)  # 保存

In [40]:
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

# 折角計算したのでpickleに保存
import pickle
with open('./data/corpus_tfidf.dump', mode='wb') as f:
    pickle.dump(corpus_tfidf, f)

# 次回からはロードできます
# with open('./data/corpus_tfidf.dump', mode='rb') as f:
#     corpus_tfidf = pickle.load(f)

In [61]:
tfidf.idfs

{0: 2.682005934324473,
 1: 9.36850646150769,
 2: 9.36850646150769,
 3: 9.36850646150769,
 4: 0.41140441994540544,
 5: 0.38551288681338153,
 6: 1.3628819123138136,
 7: 9.36850646150769,
 8: 4.010954456889608,
 9: 7.3685064615076925,
 10: 2.5356164473429503,
 11: 5.1985814600653795,
 12: 1.4556171252777301,
 13: 2.0556235062233363,
 14: 9.36850646150769,
 15: 9.36850646150769,
 16: 2.739149841428082,
 17: 9.36850646150769,
 18: 4.2810436202573525,
 19: 2.4027221768456046,
 20: 3.813917609830055,
 21: 5.2810436202573525,
 22: 4.976189038728932,
 23: 5.2810436202573525,
 24: 7.3685064615076925,
 25: 1.3024172710499193,
 26: 5.7835439607865355,
 27: 2.3024172710499196,
 28: 5.909074842870395,
 29: 9.36850646150769,
 30: 9.36850646150769,
 31: 7.04657836662033,
 32: 8.368506461507693,
 33: 6.04657836662033,
 34: 9.36850646150769,
 35: 5.7835439607865355,
 36: 9.36850646150769,
 37: 6.7835439607865355,
 38: 7.7835439607865355,
 39: 1.6542609438415692,
 40: 1.431868522505121,
 41: 4.0109544568

In [41]:
# 文書量にもよりますが、数時間程度かかったりもします。
# '18/12/03追記: LdaMulticore で worker の数を増やせばかなり早くなるかもです
lda = models.LdaModel(corpus=corpus_tfidf, id2word=dic,
                             num_topics=50, minimum_probability=0.001,
                             passes=20, update_every=0, chunksize=10000)
lda.save('./data/lda.model')  # 保存
#lda = gensim.models.LdaModel.load('./data/lda.model')  # 次回からロード可

In [44]:
lda.print_topic(10)

'0.028*"ましょ" + 0.025*"う" + 0.022*"できるかな" + 0.015*"わたし" + 0.014*"ラッキー" + 0.013*"「" + 0.010*"いっしょ" + 0.010*"わけ" + 0.010*"今年" + 0.010*"」"'

In [42]:
for i in range(10):
    print('tpc_{0}: {1}'.format(i, lda.print_topic(i)))

tpc_0: 0.039*"あ" + 0.030*"その" + 0.013*"やっぱり" + 0.012*"宝物" + 0.010*"いつまでも" + 0.010*"どんなに" + 0.009*"安心" + 0.009*"する" + 0.009*"言わ" + 0.008*"ふんわり"
tpc_1: 0.056*"ま" + 0.050*"すっ" + 0.015*"緊張" + 0.015*"温泉" + 0.014*"お" + 0.013*"!" + 0.013*"かな" + 0.012*"お待ち" + 0.012*"お仕事" + 0.012*"あっ"
tpc_2: 0.021*"ぇい" + 0.014*"月" + 0.009*"つぎ" + 0.009*"トランプ" + 0.009*"カード" + 0.008*"写真" + 0.008*"様" + 0.008*"幸運" + 0.007*"お" + 0.007*"ー"
tpc_3: 0.024*"よう" + 0.023*"なん" + 0.020*"いつか" + 0.015*"席" + 0.011*"ちゃんと" + 0.010*"お土産" + 0.010*"大阪" + 0.010*"負け" + 0.008*"姿" + 0.008*"集まり"
tpc_4: 0.032*"この" + 0.024*"場所" + 0.017*"これから" + 0.016*"けど" + 0.015*"あったかい" + 0.012*"思い" + 0.012*"だ" + 0.011*"泣い" + 0.011*"か" + 0.010*"たら"
tpc_5: 0.030*"風" + 0.019*"もこもこ" + 0.016*"夢の中" + 0.015*"のに" + 0.014*"行き" + 0.012*"吹い" + 0.012*"熱い" + 0.010*"夏" + 0.010*"碧" + 0.008*"なんだか"
tpc_6: 0.015*"これ" + 0.015*"プロデュース" + 0.014*"輝い" + 0.010*"闘う" + 0.010*"牛" + 0.010*"ぎさ" + 0.010*"もらう" + 0.010*"一緒に" + 0.010*"旅" + 0.010*"たくさん"
tpc_7: 0.016*"メリークリスマス" + 0.016*"もし